In [1]:
import tensorflow as tf
import numpy as np
import cv2

In [2]:
img_size = 256
# Placeholders for input data and the targets
X_input = tf.placeholder(tf.float32, (None, img_size, img_size, 3), name='Input')
X_input_mask = tf.placeholder(tf.float32, (None, img_size, img_size, 1), name='Input_bw')
X_latent_vector = tf.placeholder(tf.float32, (None, img_size, img_size, 1), name='Input_bw')
X_target = tf.placeholder(tf.float32, (None, img_size, img_size, 3), name='Target')

In [3]:
class vgg16_RGB:
    def __init__(self, imgs, weights=None, sess=None):
        self.imgs = imgs
        self.convlayers()
#         self.fc_layers()
        self.map = self.pool5
#         self.probs = tf.nn.softmax(self.fc3l)
        if weights is not None and sess is not None:
            self.load_weights(weights, sess)


    def convlayers(self):
        self.parameters = []

        # zero-mean input
        with tf.name_scope('preprocess') as scope:
            mean = tf.constant([123.68, 116.779, 103.939], dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean')
            images = self.imgs-mean

        # conv1_1
        with tf.name_scope('conv1_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 3, 64], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv1_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv1_2
        with tf.name_scope('conv1_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 64], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv1_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv1_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool1
        self.pool1 = tf.nn.max_pool(self.conv1_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool1')

        # conv2_1
        with tf.name_scope('conv2_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv2_2
        with tf.name_scope('conv2_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 128], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv2_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool2
        self.pool2 = tf.nn.max_pool(self.conv2_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool2')

        # conv3_1
        with tf.name_scope('conv3_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv3_2
        with tf.name_scope('conv3_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv3_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv3_3
        with tf.name_scope('conv3_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv3_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool3
        self.pool3 = tf.nn.max_pool(self.conv3_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool3')

        # conv4_1
        with tf.name_scope('conv4_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool3, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv4_2
        with tf.name_scope('conv4_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv4_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv4_3
        with tf.name_scope('conv4_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv4_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool4
        self.pool4 = tf.nn.max_pool(self.conv4_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')

        # conv5_1
        with tf.name_scope('conv5_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool4, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv5_2
        with tf.name_scope('conv5_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv5_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv5_3
        with tf.name_scope('conv5_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv5_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool5
        self.pool5 = tf.nn.max_pool(self.conv5_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')

#     def fc_layers(self):
#         # fc1
#         with tf.name_scope('fc1') as scope:
#             shape = int(np.prod(self.pool5.get_shape()[1:]))
#             fc1w = tf.Variable(tf.truncated_normal([shape, 4096],
#                                                          dtype=tf.float32,
#                                                          stddev=1e-1), name='weights')
#             fc1b = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),
#                                  trainable=True, name='biases')
#             pool5_flat = tf.reshape(self.pool5, [-1, shape])
#             fc1l = tf.nn.bias_add(tf.matmul(pool5_flat, fc1w), fc1b)
#             self.fc1 = tf.nn.relu(fc1l)
#             self.parameters += [fc1w, fc1b]

#         # fc2
#         with tf.name_scope('fc2') as scope:
#             fc2w = tf.Variable(tf.truncated_normal([4096, 4096],
#                                                          dtype=tf.float32,
#                                                          stddev=1e-1), name='weights')
#             fc2b = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),
#                                  trainable=True, name='biases')
#             fc2l = tf.nn.bias_add(tf.matmul(self.fc1, fc2w), fc2b)
#             self.fc2 = tf.nn.relu(fc2l)
#             self.parameters += [fc2w, fc2b]

#         # fc3
#         with tf.name_scope('fc3') as scope:
#             fc3w = tf.Variable(tf.truncated_normal([4096, 1000],
#                                                          dtype=tf.float32,
#                                                          stddev=1e-1), name='weights')
#             fc3b = tf.Variable(tf.constant(1.0, shape=[1000], dtype=tf.float32),
#                                  trainable=True, name='biases')
#             self.fc3l = tf.nn.bias_add(tf.matmul(self.fc2, fc3w), fc3b)
#             self.parameters += [fc3w, fc3b]

    def load_weights(self, weight_file, sess):
        weights = np.load(weight_file)
        keys = sorted(weights.keys())
        for i, k in enumerate(keys):
            print(i, k, np.shape(weights[k]))
            sess.run(self.parameters[i].assign(weights[k]))

In [4]:
class vgg16_Mask:
    def __init__(self, imgs, weights=None, sess=None):
        self.imgs = imgs
        self.convlayers()
#         self.fc_layers()
        self.map = self.pool5
#         self.probs = tf.nn.softmax(self.fc3l)
        if weights is not None and sess is not None:
            self.load_weights(weights, sess)


    def convlayers(self):
        self.parameters = []

        # zero-mean input
        with tf.name_scope('preprocess') as scope:
            mean = tf.constant([123.68, 116.779, 103.939], dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean')
            images = self.imgs-mean

        # conv1_1
        with tf.name_scope('conv1_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 3, 64], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            #[3, 3, 3, 64] represents [3, 3] filter size and 3 input channels, 64 output channels
            conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv1_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv1_2
        with tf.name_scope('conv1_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 64], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv1_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv1_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool1
        self.pool1 = tf.nn.max_pool(self.conv1_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool1')

        # conv2_1
        with tf.name_scope('conv2_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv2_2
        with tf.name_scope('conv2_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 128], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv2_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool2
        self.pool2 = tf.nn.max_pool(self.conv2_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool2')

        # conv3_1
        with tf.name_scope('conv3_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv3_2
        with tf.name_scope('conv3_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv3_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv3_3
        with tf.name_scope('conv3_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv3_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool3
        self.pool3 = tf.nn.max_pool(self.conv3_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool3')

        # conv4_1
        with tf.name_scope('conv4_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool3, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv4_2
        with tf.name_scope('conv4_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv4_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv4_3
        with tf.name_scope('conv4_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv4_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool4
        self.pool4 = tf.nn.max_pool(self.conv4_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')

        # conv5_1
        with tf.name_scope('conv5_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool4, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv5_2
        with tf.name_scope('conv5_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv5_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv5_3
        with tf.name_scope('conv5_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv5_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool5
        self.pool5 = tf.nn.max_pool(self.conv5_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')

#     def fc_layers(self):
#         # fc1
#         with tf.name_scope('fc1') as scope:
#             shape = int(np.prod(self.pool5.get_shape()[1:]))
#             fc1w = tf.Variable(tf.truncated_normal([shape, 4096],
#                                                          dtype=tf.float32,
#                                                          stddev=1e-1), name='weights')
#             fc1b = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),
#                                  trainable=True, name='biases')
#             pool5_flat = tf.reshape(self.pool5, [-1, shape])
#             fc1l = tf.nn.bias_add(tf.matmul(pool5_flat, fc1w), fc1b)
#             self.fc1 = tf.nn.relu(fc1l)
#             self.parameters += [fc1w, fc1b]

#         # fc2
#         with tf.name_scope('fc2') as scope:
#             fc2w = tf.Variable(tf.truncated_normal([4096, 4096],
#                                                          dtype=tf.float32,
#                                                          stddev=1e-1), name='weights')
#             fc2b = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),
#                                  trainable=True, name='biases')
#             fc2l = tf.nn.bias_add(tf.matmul(self.fc1, fc2w), fc2b)
#             self.fc2 = tf.nn.relu(fc2l)
#             self.parameters += [fc2w, fc2b]

#         # fc3
#         with tf.name_scope('fc3') as scope:
#             fc3w = tf.Variable(tf.truncated_normal([4096, 1000],
#                                                          dtype=tf.float32,
#                                                          stddev=1e-1), name='weights')
#             fc3b = tf.Variable(tf.constant(1.0, shape=[1000], dtype=tf.float32),
#                                  trainable=True, name='biases')
#             self.fc3l = tf.nn.bias_add(tf.matmul(self.fc2, fc3w), fc3b)
#             self.parameters += [fc3w, fc3b]

    def load_weights(self, weight_file, sess):
        weights = np.load(weight_file)
        keys = sorted(weights.keys())
        for i, k in enumerate(keys):
            print(i, k, np.shape(weights[k]))
            sess.run(self.parameters[i].assign(weights[k]))

In [5]:
def encoder(z, reuse=False):
    """
    Encode part of the autoencoder.
    :param x: input to the autoencoder
    :param reuse: True -> Reuse the encoder variables, False -> Create or search of variables before creating
    :return: tensor which is the hidden latent variable of the autoencoder.
    """
    if reuse:
        tf.get_variable_scope().reuse_variables()
    with tf.name_scope('Encoder'):
        filters = {1:2,2:1,3:3, 4:40, 5:50, 6:60}
        conv1 = tf.layers.conv2d(z, filters[1], (3,3),strides=(1, 1), padding='valid',activation=tf.nn.relu, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        conv2 = tf.layers.conv2d(conv1, filters[1], (3,3),strides=(1,1),padding='valid',activation=tf.nn.relu,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        
#         conv = tf.concat([conv3, conv4], 3)
        #conv_shortcut = conv
        conv3 = tf.layers.conv2d(con2, filters[3],(3,3), strides=(1, 1), padding='valid', activation=tf.nn.relu,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        print('conv1',np.shape(conv1))
        print('conv2',np.shape(conv2))
        print('conv3',np.shape(conv3))
        print('conv4',np.shape(conv4))
        print('conv5',np.shape(conv5))
        lv = tf.contrib.layers.flatten(conv5)
    return conv3, lv

In [6]:
def decode(x):
    act = tf.nn.relu(x)
    upsample1 = tf.image.resize_nearest_neighbor(act, (512,512))
    conv4 = tf.layers.conv2d(upsample1, 512, (7,7), padding='same',activation=tf.nn.relu)
    upsample2 = tf.image.resize_nearest_neighbor(conv5, (636,636))
    conv5 = tf.layers.conv2d(upsample2, 256, (5,5), padding='same',activation=tf.nn.relu)
    upsample3 = tf.image.resize_nearest_neighbor(conv6, (776,776))
    conv6 = tf.layers.conv2d(upsample3, 128, (7,7), padding='same',activation=tf.nn.relu)
    upsample4 = tf.image.resize_nearest_neighbor(conv7, (1024,1024))
    conv7 = tf.layers.conv2d(upsample4, 64, (7,7), padding='same',activation=tf.nn.relu)
    upsample1 = tf.image.resize_nearest_neighbor(act, (512,512))
    conv8 = tf.layers.conv2d(upsample1, 16, (7,7), padding='same',activation=tf.nn.relu)
    upsample2 = tf.image.resize_nearest_neighbor(conv5, (636,636))
    conv9 = tf.layers.conv2d(upsample2, 3, (5,5), padding='same',activation=tf.nn.relu)
    return conv9

In [7]:
def discriminator(X,reuse=None):
    with tf.variable_scope('dis',reuse=reuse):
        conv = tf.layers.conv2d(X, 20, (5,5), padding='same',activation=tf.nn.relu)
        features = tf.contrib.layers.flatten(latent_vector_map)
        hidden1=tf.layers.dense(inputs=features,units=512,activation=tf.nn.leaky_relu)
        hidden2=tf.layers.dense(inputs=hidden1,units=256,activation=tf.nn.leaky_relu)
        hidden3=tf.layers.dense(inputs=hidden2,units=128,activation=tf.nn.leaky_relu)
        logits=tf.layers.dense(hidden3,units=1)
        output=tf.sigmoid(logits)
        
        return output,logits

In [1]:
def make_normal_distribution(input_shape):
    mu, sigma = 0.0, 1.0
    norm_dist = np.random.normal(mu, sigma, input_shape)
    count, bins, ignored = plt.hist(norm_dist, 20, normed=True)
    return norm_dist

In [8]:
def gram_matrix(input_tensor):
    # if input tensor is a 3D array of size Nh x Nw X Nc
    # we reshape it to a 2D array of Nc x (Nh*Nw)
    channels = int(input_tensor.shape[-1])
    a = tf.reshape(input_tensor, [-1, channels])
    n = tf.shape(a)[0]

    # get gram matrix 
    gram = tf.matmul(a, a, transpose_a=True)

    return gram

def get_style_loss(base_style, gram_target):
    height, width, channels = base_style.get_shape().as_list()
    gram_style = gram_matrix(base_style)
    # Original eqn as a constant to divide i.e 1/(4. * (channels ** 2) * (width * height) ** 2)
    return tf.reduce_mean(tf.square(gram_style - gram_target)) / (channels**2 * width * height) #(4.0 * (channels ** 2) * (width * height) ** 2)

In [9]:
def reconstruction_loss(x_target, reconstructed_image):
    #lr=0.001
    L2_loss = tf.reduce_mean(tf.square(x_target - reconstructed_image))
    style_loss = get_style_loss(reconstructed_image, x_target)
    reconstruction_loss = L2_loss 
    #reconstruction_optimizer = tf.train.AdamOptimizer(learning_rate=lr, beta1=0.9).minimize(reconstruction_loss)
    return reconstruction_loss, style_loss

#KL divergence loss
def KLdivergence_loss(encode_LD, groundtruth_LD, normal_dist):
    k = tf.keras.losses.KLDivergence()
    kl_loss = k(encode_LD, groundtruth_LD) + k(encode_LD, normal_dist) + k(groundtruth_LD, normal_dist)
    return tf.reduce_mean(kl_loss)

#wgan loss
def wgan_loss(d_real, d_fake):
    #lr=0.01
    disc_loss = tf.reduce_mean(d_real) - tf.reduce_mean(d_fake)
    #discriminator_optimizer = tf.train.RMSPropOptimizer(learning_rate=lr).minimize(-disc_loss)
    return disc_loss

In [ ]:
def distance_dependent_loss(reconstructed_image, groundtruth_image, count):
    arg_list = open()
    w, h = arg_list(count)
    
    #crop reconst_image to get reconst_patch
    #crop gt_image to get gt_patch
    for i in range(w):
        for j in range(h):
            if (i<j):
                gamma[i][j] = (0.99)**(i)
            else:
                gamma[i][j] = (0.99)**(j)
                
    L2_loss = (tf.square(reconstructedpatch - groundtruthpatch))
    distance_dependent_loss = tf.reduce_mean(gamma * L2_loss)
    return distance_dependent_loss

In [10]:
def load_dataset(i):
    training_label = []
    training_mask = []
    training_image = []
    for j in range((i*62),(62+(i*62))):
        image = cv2.imread('D:\Temp\data_256\h\highway/00000%d.jpg' %(j+100))
        training_label.append(image)
        x = y = randint(100, 110)
        w = h = randint(60, 65)
        print(x,y,w,h)
        image[x:(x+w),y:(y+h)] = 255
        training_image.append(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY ) 
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                if (image[i][j] >= 128):
                    image[i][j] = 255
                else:
                    image[i][j] = 0
        training_mask.append(image)
    return training_image, training_mask, training_label

In [3]:
if __name__ == '__main__':
    sess = tf.Session()
    vgg_RGB = vgg16_RGB(X_input, 'vgg16_weights.npz', sess)
    vgg_Mask = vgg16_Mask(X_input_mask, 'vgg16_weights.npz', sess)
    conv_RGB = vgg16_RGB.pool5
    conv_Mask = vgg16_Mask.pool5
    conv_concat = tf.concat([conv_RGB, conv_Mask], 3)
    latent_vector_map = encoder(conv_concat)
    latent_vector_feature = tf.contrib.layers.flatten(latent_vector_map)
    reconstructed_image = decode(latent_vector_map)
    norm_dist = make_normal_distribution(latent_vector_feature.get_shape())
    
    lambda_1 = 0.9
    lambda_2 = 0.8
    lambda_3 = 0.85
    lambda_4 = 0.88
    
    # LOSSES
    
    D_output_real,D_logits_real=discriminator(X_target)
    
    D_output_fake,D_logits_fake=discriminator(reconstructed_image,reuse=True)
    
    w_gan_loss = wgan_loss(D_output_real, D_output_fake)
    
    reconstruction_loss, style_loss = reconstruction_loss(X_target, reconstructed_image)
    
    discounted_reconstruction_loss = distance_dependent_loss(reconstructed_image, X_target, counter)
    
    KLdivergence_loss = KLdivergence_loss(latent_vector_feature, X_latent_vector, norm_dist)
    
    total_loss = (lambda_1*reconstruction_loss) + (lambda_2*KLdivergence_loss) + (lambda_3*style_loss) - (lambda_4*w_gan_loss)
    
    reconstruction_optimizer = tf.train.AdamOptimizer(learning_rate=0.001,
                                                       beta1=0.9).minimize(total_loss)
    
    batch_size=62
    epochs=100
    num_batches=56
    init=tf.global_variables_initializer()
    samples_n = [] 
    minibatch_loss_n = 0
    cost = []
    with tf.Session() as sess:
        sess.run(init)
        for epoch in tqdm(range(epochs)):
            count=0
            for i in range(num_batches):
                X, X_mask, Y = load_dataset(i)
                _, temp_loss = sess.run([reconstruction_optimizer,total_loss, count], feed_dict={X_input:X, X_input_mask:X_mask, X_target:Y, })
#                 _, temp_loss = sess.run([disc_opt,disc_loss], feed_dict={x_input:X, x_input_bw:X_bw, x_input_mask:X_mask})
                minibatch_loss_n += temp_loss / num_batches
                count+=1
        
            print("on epoch{}".format(epoch))
            # Print the cost every epoch
            print ("Cost after epoch %i: %f" % (epoch, minibatch_loss_n))
            cost_n.append(minibatch_loss_n)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\bhatt\desktop\jupyter notebook\env\lib\site-packages\IPython\core\interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-1eb1dfaaf490>", line 2, in <module>
    sess = tf.Session()
NameError: name 'tf' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\bhatt\desktop\jupyter notebook\env\lib\site-packages\IPython\core\interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\bhatt\desktop\jupyter notebook\env\lib\site-packages\IPython\core\ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "c:

NameError: name 'tf' is not defined